In [16]:
# EH:  Import the required libraries and dependencies
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from dotenv import find_dotenv
from pathlib import Path
import hvplot.pandas
import holoviews as hv
from pandas.tseries.offsets import QuarterEnd



#Data Gathering

In [17]:
#EH: import stock pool list

stockpool_list_df=pd.read_csv(
    "Resources/sector_list_v2.txt",
    sep=';'
)

#EH: check blank count
stockpool_list_df.isnull().sum() 

Sector     0
Ticker     0
Company    0
dtype: int64

In [18]:
#EH: review stockpool dataframe
stockpool_list_df.rename(columns={'Ticker':'ticker'},inplace=True)
stockpool_list_df.head()

,Sector,ticker,Company
0,Commercial Services,SPGI,S&P Global Inc.
1,Commercial Services,MCO,Moody’s Corporation
2,Commercial Services,CPRT,"Copart, Inc."
3,Commercial Services,EFX,"Equifax, Inc."
4,Commercial Services,FLT,"FleetCor Technologies, Inc."


In [19]:
#EH: list of sector
sector_list=list(stockpool_list_df['Sector'].unique())
sector_list

['Commercial Services',
 'Communications',
 'Consumer Durables',
 'Consumer Non-Durables',
 'Consumer Services',
 'Distribution Services',
 'Electronic Technology',
 'Energy Minerals',
 'Finance',
 'Health Services',
 'Health Technology',
 'Industrial Services',
 'Non-Energy Minerals',
 'Process Industries',
 'Producer Manufacturing',
 'Retail Trade',
 'Technology Services',
 'Transportation',
 'Utilities']

In [20]:
#EH: add column to join company name with ticker

stockpool_list_df['Company_Ticker']=stockpool_list_df['Company']+' ('+stockpool_list_df['ticker']+')'
stockpool_list_df

,Sector,ticker,Company,Company_Ticker
0,Commercial Services,SPGI,S&P Global Inc.,S&P Global Inc. (SPGI)
1,Commercial Services,MCO,Moody’s Corporation,Moody’s Corporation (MCO)
2,Commercial Services,CPRT,"Copart, Inc.","Copart, Inc. (CPRT)"
3,Commercial Services,EFX,"Equifax, Inc.","Equifax, Inc. (EFX)"
4,Commercial Services,FLT,"FleetCor Technologies, Inc.","FleetCor Technologies, Inc. (FLT)"
...,...,...,...,...
484,Utilities,LNT,Alliant Energy Corporation,Alliant Energy Corporation (LNT)
485,Utilities,ATO,Atmos Energy Corporation,Atmos Energy Corporation (ATO)
486,Utilities,NI,NiSource Inc,NiSource Inc (NI)
487,Utilities,NRG,"NRG Energy, Inc.","NRG Energy, Inc. (NRG)"


In [21]:
#EH:  create list for company ticker list

company_ticker_list=stockpool_list_df['Company_Ticker']
company_ticker_list

0                       S&P Global Inc. (SPGI)
1                    Moody’s Corporation (MCO)
2                          Copart, Inc. (CPRT)
3                          Equifax, Inc. (EFX)
4            FleetCor Technologies, Inc. (FLT)
                        ...                   
484           Alliant Energy Corporation (LNT)
485             Atmos Energy Corporation (ATO)
486                          NiSource Inc (NI)
487                     NRG Energy, Inc. (NRG)
488    Pinnacle West Capital Corporation (PNW)
Name: Company_Ticker, Length: 489, dtype: object

In [22]:
#EH: load env file
load_dotenv()


True

In [23]:

#EH: Find env file
find_dotenv()

'C:\\Users\\eunic\\onedrive\\desktop\\FinTech-Workspace\\Project2\\.env'

In [24]:
#EH:  check alpha key

alpha_api_key=os.getenv('ALPHA_API_KEY')
print(type(alpha_api_key))

<class 'str'>


In [25]:
#EH:  create stock close price df from Alpha data

def alpha_daily_data(stock, api, days_of_record):
    data_url="https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=" + stock + "&outputsize=full&apikey=" + api
    r = requests.get(data_url)
    data = r.json()
    df=pd.DataFrame(data['Time Series (Daily)']).T.rename(columns={'1. open':'open','2. high':'high','3. low':'low','4. close':'close','5. volume':'volume'})
    df=df.reset_index()
    df['index']=pd.to_datetime(df['index'])
    df['QEdate']=pd.to_datetime(df['index'],format='%Y%m')+QuarterEnd(1)
    df=df.set_index('index')
    df.index.name='date'
    df.index=pd.to_datetime(df.index)
    df['close']=df['close'].astype('float64')
    df=df.iloc[:days_of_record]
    df.drop(columns=['open','high','low','volume'],inplace=True)
    


    return df

In [26]:
#EH: Compile QE_ROI function of a ticker


def roi_compile(df):
  qe_list=list(df['QEdate'].unique())
  qe_roi_df=pd.DataFrame()
  for num in range(len(qe_list)):
    qe_roi_df=qe_roi_df.append(df[df['QEdate']==qe_list[num]].iloc[0])
    qe_roi_df['q_roi']=qe_roi_df['close'].pct_change(periods=-1)

  
  return qe_roi_df



In [28]:
# EH: import cashflow function


def cash(stock):
    cash_url='https://www.alphavantage.co/query?function=CASH_FLOW&symbol='+stock+'&apikey='+ alpha_api_key
    r = requests.get(cash_url)
    data = r.json()
    cash_compile=pd.DataFrame()
    for num in range(len(data['quarterlyReports'])):
        ind=list(data['quarterlyReports'][num].keys())
        cash_df=pd.DataFrame(data['quarterlyReports'][num],index=ind)
        cash_df.reset_index(inplace=True,drop=True)
        cash_df=cash_df.iloc[0]
        cash_df['ticker']=stock
        cash_compile=cash_compile.append(cash_df)
        cash_compile['fiscalDateEnding']=pd.to_datetime(cash_compile['fiscalDateEnding'])
    
    cash_compile.set_index('fiscalDateEnding',inplace=True)

    return cash_compile


In [29]:
#EH: Get EPS function
def eps(stock):
    eps_url='https://www.alphavantage.co/query?function=EARNINGS&symbol='+stock+'&apikey='+ alpha_api_key
    r = requests.get(eps_url)
    data = r.json()
    # return data['quarterlyEarnings']
    eps_compile=pd.DataFrame()
    for num in range(len(data['quarterlyEarnings'])):
        key_list=data['quarterlyEarnings'][num].keys()
        eps_df=pd.DataFrame(data['quarterlyEarnings'][num],index=key_list)
        eps_df.reset_index(drop=True,inplace=True)
        eps_df=eps_df.iloc[0]
        eps_df=eps_df.drop(columns=['reportedDate'])
        eps_df['ticker']=stock
        eps_compile=eps_compile.append(eps_df)
    eps_compile.set_index('fiscalDateEnding',inplace=True)
    eps_compile=eps_compile.drop(['reportedDate'],axis=1)


    return eps_compile

In [30]:
# EH:  get inflation

inflation_url='https://www.alphavantage.co/query?function=INFLATION_EXPECTATION&apikey='+ alpha_api_key
r= requests.get(inflation_url)
data = r.json()

inflation=pd.DataFrame()
for num in range(len(data['data'])):

    inflation_df=pd.DataFrame(data['data'][num],index=['date','value'])
    inflation_df=inflation_df.iloc[0]
    inflation=inflation.append(inflation_df)
    
inflation.head()

,date,value
date,2022-02-01,4.9
date,2022-01-01,4.9
date,2021-12-01,4.8
date,2021-11-01,4.9
date,2021-10-01,4.8


In [31]:
#EH:   get Consumer sentiment

consumer_s_url='https://www.alphavantage.co/query?function=CONSUMER_SENTIMENT&apikey='+ alpha_api_key
r= requests.get(consumer_s_url)
data = r.json()

consumer_s=pd.DataFrame()
for num in range(len(data['data'])):

    consumer_s_df=pd.DataFrame(data['data'][num],index=['date','value'])
    consumer_s_df=consumer_s_df.iloc[0]
    consumer_s=consumer_s.append(consumer_s_df)
consumer_s.head()

,date,value
date,2022-02-01,62.8
date,2022-01-01,67.2
date,2021-12-01,70.6
date,2021-11-01,67.4
date,2021-10-01,71.7


In [32]:
#EH: inflation df for chart

inflation_date=inflation.set_index('date')
inflation_date.index=pd.to_datetime(inflation_date.index)
#EH: update date to closest month end
inflation_date.index=inflation_date.index.shift(-1,freq='D')
inflation_date.head()

,value
date,
2022-01-31,4.9
2021-12-31,4.9
2021-11-30,4.8
2021-10-31,4.9
2021-09-30,4.8


In [34]:
#EH: inflation Charts


inflation_date['2012-02-01':].hvplot(title='Inflation Chart',width=1000,height=500)

C:\Users\eunic\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  after removing the cwd from sys.path.


:Curve   [date]   (value)

In [37]:
#EH: consumer sentiment df for chart

consumer_date=consumer_s.set_index('date')
consumer_date.index=pd.to_datetime(consumer_date.index)
#EH: update date to closest month end
consumer_date.index=consumer_date.index.shift(-1,freq='D')
consumer_date.head()

,value
date,
2022-01-31,62.8
2021-12-31,67.2
2021-11-30,70.6
2021-10-31,67.4
2021-09-30,71.7


In [38]:
#EH: consumer sentiment Charts



consumer_date['2017-02-01':].hvplot(width=1000,height=500, ylabel='consumer indiator',title='consumer indicator')

C:\Users\eunic\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  """


:Curve   [date]   (value)

In [39]:
#EH: function to reset index and to_datetime

def resetidx_todate(df):
  df=df.reset_index()
  df['date']=pd.to_datetime(df['date'])
  return df

In [40]:
#EH: reset index/to_datetime for inflation df

inflation_prep=resetidx_todate(inflation_date)
inflation_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    530 non-null    datetime64[ns]
 1   value   530 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 8.4+ KB


In [41]:
#EH: reset index/to_datetime for consumer df
consumer_prep=resetidx_todate(consumer_date)
consumer_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 832 entries, 0 to 831
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    832 non-null    datetime64[ns]
 1   value   832 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 13.1+ KB


In [42]:
sector_list

['Commercial Services',
 'Communications',
 'Consumer Durables',
 'Consumer Non-Durables',
 'Consumer Services',
 'Distribution Services',
 'Electronic Technology',
 'Energy Minerals',
 'Finance',
 'Health Services',
 'Health Technology',
 'Industrial Services',
 'Non-Energy Minerals',
 'Process Industries',
 'Producer Manufacturing',
 'Retail Trade',
 'Technology Services',
 'Transportation',
 'Utilities']

In [43]:
#EH: create stock list for specific sector
#EH:  Cannot loop through sector to compile data/generate output due to API calls limitation per day.

stock_list=list(stockpool_list_df[stockpool_list_df['Sector']=='Health Technology']['ticker'])

#EH:  print number of tickers per sector
num_of_stock=len(stock_list)
num_of_stock

43

In [44]:
#EH:  Create URL List

url_list = []
for each in stock_list:
  data_url='https://www.alphavantage.co/query?function=CASH_FLOW&symbol='+each+'&apikey='+ alpha_api_key
  url_list.append(data_url)

In [ ]:
# EH:  List of urls to check ticker data error


for url in url_list:
    try:
        response = requests.get(url)

        # Check Status Codes
        if response.status_code == 200:
            data=response.json()
            #EH: print the earliest date of quarter end data
            print(data["quarterlyReports"][-1]["fiscalDateEnding"])
            print('================')
        else:
            print(f"The url: {url} is invalid.\nResponse code: {response.status_code}")

    except Exception as error:
        # If an exception occurs in the try portion, the code in this branch will be executed.
        print(f"Unable to connect to the server at url: {url}.\nError:\n{error}")
    finally:
        # Everything in finally will be executed regardless of try/except
        print(f"Parsed url: {url}\n")

In [46]:
#EH: Compile ROI for multiple tickers
stock_df_list=[]
for num in range(num_of_stock):
    close_df=alpha_daily_data(stock_list[num], alpha_api_key, 2000)
    roi_df=roi_compile(close_df)
    stock_df_list.append(roi_df)
join_roi=pd.concat(stock_df_list,axis=0,keys=stock_list)
join_roi=join_roi.reset_index()
join_roi.columns=['ticker','date','close','QEdate','q_roi']
join_roi.set_index('QEdate',inplace=True)
    

In [47]:
#EH: Compile cashflow
consolidate_cash=pd.DataFrame()
for each_sl1 in stock_list:

    ticker_cash=cash(each_sl1)
    consolidate_cash=consolidate_cash.append(ticker_cash)

In [48]:
#EH:  Compile EPS
eps_consolidate=pd.DataFrame()
for each_sl2 in stock_list:
    eps_tickergroup=eps(each_sl2)
    eps_consolidate=eps_consolidate.append(eps_tickergroup)

# Merge Data

In [49]:
#EH: Merge sector, cashflow, eps, inflation, consumer sentiment

#stockpool_list_df on ticker
#join_price on date/ticker
#consolidate_cash on date/ticker
#eps_consolidate on date/ticker
#inflation_date on date
#consumer_date on date



#EH: Merge cash and sector 
consolidate_cash=consolidate_cash.reset_index()
merge_cash_sector=pd.merge(consolidate_cash,stockpool_list_df[['ticker','Sector']],on='ticker',how='left')
merge_cash_sector.rename(columns={'fiscalDateEnding':'date'},inplace=True)
    
#EH: Merge cash,sector,eps
eps_consolidate=eps_consolidate.reset_index()
eps_consolidate.rename(columns={'fiscalDateEnding':'date'},inplace=True)
eps_consolidate['date']=pd.to_datetime(eps_consolidate['date'])
merge_cash_sector_eps=pd.merge(merge_cash_sector,eps_consolidate.loc[:,'date':'ticker'],on=['date','ticker'],how='left')
    
#EH: Merge cash,sector,eps,inflation
merge_cash_sector_eps_inf=pd.merge(merge_cash_sector_eps,inflation_prep,on='date',how='left')
merge_cash_sector_eps_inf.rename(columns={'value':'inflation'},inplace=True)
    
#EH: Merge cash,sector,eps,inflation,consumer sentiment
merge_cash_sector_eps_inf_cs=pd.merge(merge_cash_sector_eps_inf,consumer_prep,on='date',how='left')
merge_cash_sector_eps_inf_cs.rename(columns={'value':'cs_sentiment'},inplace=True)

#EH: Merge cash,sector,eps,inflation,consumer sentiment, ROI    
merge_cash_sector_eps_inf_cs_roi=pd.merge(merge_cash_sector_eps_inf_cs,join_roi,left_on=['date','ticker'],right_on=['QEdate','ticker'],how='left')
merge_cash_sector_eps_inf_cs_roi.drop(columns=['date_y'],inplace=True)
merge_cash_sector_eps_inf_cs_roi=merge_cash_sector_eps_inf_cs_roi.replace('None',0)
merge_cash_sector_eps_inf_cs_roi=merge_cash_sector_eps_inf_cs_roi.dropna()

In [50]:
#EH:  update datatype to float
merge_cash_sector_eps_inf_cs_roi.loc[:,
    [
     'operatingCashflow',
     'paymentsForOperatingActivities',
     'changeInOperatingLiabilities',
     'changeInOperatingAssets',
     'depreciationDepletionAndAmortization',
     'capitalExpenditures',
     'changeInReceivables',
     'changeInInventory',
     'profitLoss',
     'cashflowFromInvestment',
     'cashflowFromFinancing',
     'proceedsFromRepaymentsOfShortTermDebt',
     'paymentsForRepurchaseOfCommonStock',
     'paymentsForRepurchaseOfEquity',
     'dividendPayout',
     'dividendPayoutCommonStock',
     'proceedsFromIssuanceOfCommonStock',
     'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet',
     'proceedsFromRepurchaseOfEquity',
     'changeInCashAndCashEquivalents',
     'changeInExchangeRate',
     'netIncome',
     'reportedEPS',
     'estimatedEPS',
     'surprise',
     'surprisePercentage',
     'inflation',
     'cs_sentiment']]=merge_cash_sector_eps_inf_cs_roi.loc[:,
    [
     'operatingCashflow',
     'paymentsForOperatingActivities',
     'changeInOperatingLiabilities',
     'changeInOperatingAssets',
     'depreciationDepletionAndAmortization',
     'capitalExpenditures',
     'changeInReceivables',
     'changeInInventory',
     'profitLoss',
     'cashflowFromInvestment',
     'cashflowFromFinancing',
     'proceedsFromRepaymentsOfShortTermDebt',
     'paymentsForRepurchaseOfCommonStock',
     'paymentsForRepurchaseOfEquity',
     'dividendPayout',
     'dividendPayoutCommonStock',
     'proceedsFromIssuanceOfCommonStock',
     'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet',
     'proceedsFromRepurchaseOfEquity',
     'changeInCashAndCashEquivalents',
     'changeInExchangeRate',
     'netIncome',
     'reportedEPS',
     'estimatedEPS',
     'surprise',
     'surprisePercentage',
     'inflation',
     'cs_sentiment']].astype('float64')

In [51]:
#EH:  Export compiled data per sector
#EH:  Cannot loop through sector to compile data/generate output due to API calls limitation per day.
merge_cash_sector_eps_inf_cs_roi.to_csv('Data_Prep_Output/Health Technology.csv')